In [1]:
from src.embeddings.car_make import CarMakeEmbedding
from datetime import date
import pandas as pd

## Analytical sandbox

We read the dataset with the following columns:
'person_id', 'person_age', 'person_sex', 'vehicle_make', 'vehicle_year', 'overall_rating'
From these columns and as we want to analyze the vehicle safety rating, we will delete the information related to the driver, so we will delete 'person_id', 'person_age', 'person_sex'. More specifically, we will try to find a relation between makes + years ~ safety. Looking for a pattern in this variables and computing clustering or regressions. 

In [4]:
df = pd.read_csv(
    "/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/ui/sample_tables/mvc_safety_rating_by_accidents.csv"
)
df = df[["vehicle_make", "vehicle_year", "overall_rating"]]

## Feature engineering

In [41]:
df["vehicle_age"] = date.today().year - df["vehicle_year"]
df["norm_overall_rating"] = df["overall_rating"] / 5

In [42]:
df["vehicle_make_embedding"] = df["vehicle_make"].apply(lambda x: cme.execute(x)[0])
make_df = pd.DataFrame(
    df["vehicle_make_embedding"].tolist(),
    columns=[f"make_{i}" for i in range(len(df["vehicle_make_embedding"][0]))],
)
df = pd.concat([df.drop(columns=["vehicle_make_embedding"]), make_df], axis=1)
df.dropna(inplace=True)

In [43]:
df = df[[f"make_{i}" for i in range(10)] + ["vehicle_age", "overall_rating"]]

In [44]:
X, y = df[[f"make_{i}" for i in range(10)] + ["vehicle_age"]], df["overall_rating"]

In [45]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.15142865825553686

In [49]:
from sklearn.linear_model import Ridge
import numpy as np

clf = Ridge(alpha=1.0)
clf.fit(X, y)
clf.score(X, y)

0.15137167725163359

In [50]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=0.1)
clf.fit(X, y)
clf.score(X, y)

0.0719619526236357

In [52]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor()
regr.fit(X, y)
regr.score(X, y)

0.21709028006879993